In [ ]:
## Access the twitter api to download a bunch of tweets about san fransisco. I ran this from the command line to pipe data to a file

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

print 'Begin script...'

consumer_key = "IyhOZXYvlps1ABWB6UWMBmVsh"
consumer_secret =  "SIFfJp78RCcBS3JmECj7ZS4MuJ8apYqtNMbAC3Zy7xd2Szg5Wq"
access_token = '1411256130-wJk78NyXkZLJlPVsGCHq8mgtX0wHcDanc73fy0k'
access_token_secret = 'sNPVmByioAhvlcQXHVmg2wd5lU3RJZF5drDcrsx0kaOCi'

url = 'https://api.twitter.com/1.1/'


#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):
 
    def on_data(self, data):
        print data
        return True

    def on_error(self, status):
        print status

    #This handles Twitter authetification and the connection to Twitter Streaming API
mylistener = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
mystream = Stream(auth, mylistener)

print 'starting...'
mystream.filter(track=['sanfrancisco'])